<a href="https://colab.research.google.com/github/shrybhrdwj/TheCodeStreak/blob/main/BrainTumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import zipfile
import os
import matplotlib.pyplot as plt

# Path to the zip file containing brain tumor images
zip_file_path = 'brain_tumor_dataset.zip'  # Replace with your zip file path
output_folder = 'enhanced_images'

# Create an output folder to save processed images if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Step 1: Unzip the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all images to a temporary directory
    zip_ref.extractall(output_folder)

# Step 2: Process each image in the extracted folder
# Get a list of all image files in the output folder
image_files = [f for f in os.listdir(output_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_name in image_files:
    image_path = os.path.join(output_folder, image_name)

    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded correctly
    if image is not None:
        # Apply a median filter with a kernel size of 5
        enhanced_image = cv2.medianBlur(image, 5)

        # Save the enhanced image
        enhanced_image_path = os.path.join(output_folder, f'enhanced_{image_name}')
        cv2.imwrite(enhanced_image_path, enhanced_image)

        # Optionally, display the original and enhanced images
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(image, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title("Enhanced Image (Median Filtered)")
        plt.imshow(enhanced_image, cmap='gray')
        plt.axis('off')

        plt.show()
    else:
        print(f"Error: Could not load {image_name}")


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to the images and labels (assuming binary classification: tumor vs. no tumor)
data_folder = 'enhanced_images'  # Folder with enhanced images
image_size = (128, 128)  # Resize images to a common size
categories = ['tumor', 'no_tumor']  # Folders should be named according to categories

# Load and preprocess data
images = []
labels = []

for label, category in enumerate(categories):
    category_path = os.path.join(data_folder, category)
    for filename in os.listdir(category_path):
        file_path = os.path.join(category_path, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Read in grayscale
        if image is not None:
            image = cv2.resize(image, image_size)  # Resize to common size
            images.append(image)
            labels.append(label)

# Convert lists to numpy arrays and normalize
images = np.array(images).reshape(-1, image_size[0], image_size[1], 1) / 255.0
labels = to_categorical(np.array(labels), num_classes=len(categories))  # One-hot encode labels

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(rotation_range=15, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(len(categories), activation='softmax')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    validation_data=(X_test, y_test), epochs=epochs)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Save the model
model.save("brain_tumor_classifier.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.4774 - loss: 0.8653 - val_accuracy: 0.7451 - val_loss: 0.6918
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.5226 - loss: 0.6889 - val_accuracy: 0.5294 - val_loss: 0.6853
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6445 - loss: 0.6515 - val_accuracy: 0.5294 - val_loss: 0.7224
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6261 - loss: 0.6668 - val_accuracy: 0.5294 - val_loss: 0.6706
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6504 - loss: 0.6167 - val_accuracy: 0.5294 - val_loss: 0.6569
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.6198 - loss: 0.6214 - val_accuracy: 0.5294 - val_loss: 0.6677
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6494 - loss: 0.5907 - val_accuracy: 0.5490 - val_loss: 0.6223
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6535 - loss: 0.5988 - val_accuracy: 0.5294 - val_loss: 0.6364
Epoch 9/10
7

Test Accuracy: 0.57


In [ ]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

In [22]:
import zipfile
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import BatchNormalization

# Step 1: Unzip the dataset
def unzip_dataset(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Dataset extracted to {extract_path}")

# Unzip the brain tumor images
zip_path = 'brain_tumor_images.zip'  # Path to your zipped dataset
extract_path = 'brain_tumor_images'  # Path where the dataset will be extracted
unzip_dataset(zip_path, extract_path)

# Step 2: Apply Median Filter for image enhancement
def apply_median_filter_to_images(input_dir, output_dir, filter_size=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Process each folder inside the input directory (glioma, meningioma, pituitary, no_tumor)
    for class_name in os.listdir(input_dir):
        class_folder = os.path.join(input_dir, class_name)

        # Skip files that are not directories (e.g., test/train folders)
        if not os.path.isdir(class_folder):
            continue

        class_output_folder = os.path.join(output_dir, class_name)
        if not os.path.exists(class_output_folder):
            os.makedirs(class_output_folder)

        # Process each image in the class folder
        for image_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_name)

            # Read and apply median filter
            image = cv2.imread(image_path)
            filtered_image = cv2.medianBlur(image, filter_size)

            # Save the enhanced image
            output_image_path = os.path.join(class_output_folder, image_name)
            cv2.imwrite(output_image_path, filtered_image)
            print(f"Processed {image_name} with median filter")

# Apply median filter to training images
input_dir = os.path.join(extract_path, 'Training')
output_dir = os.path.join(extract_path, 'training_filtered')
apply_median_filter_to_images(input_dir, output_dir)

# Apply median filter to testing images
input_dir = os.path.join(extract_path, 'Testing')
output_dir = os.path.join(extract_path, 'testing_filtered')
apply_median_filter_to_images(input_dir, output_dir)

# Step 3: Prepare data generators for training and testing
image_size = (224, 224)  # VGG16 expects images of size 224x224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255,           # Normalize pixel values
    rotation_range=15,         # Randomly rotate images
    width_shift_range=0.1,     # Shift images horizontally
    height_shift_range=0.1,    # Shift images vertically
    zoom_range=0.1,            # Randomly zoom images
    horizontal_flip=True,      # Randomly flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load the filtered training and testing datasets
train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(extract_path, 'training_filtered'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multiclass classification
)

test_generator = test_datagen.flow_from_directory(
    directory=os.path.join(extract_path, 'testing_filtered'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multiclass classification
)

# Step 4: Load the VGG16 model (without the top classification layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# Step 5: Build the complete model with a custom classifier
model = Sequential([
    base_model,                # VGG16 as feature extractor
    Flatten(),                 # Flatten the output of VGG16
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),  # Dense layer for learning high-level features
    Dropout(0.5),              # Dropout for regularization
    Dense(4, activation='softmax')  # Output layer for 4 classes (glioma, meningioma, pituitary, no_tumor)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the model (only the custom layers)
epochs = 10
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs
)

# Unfreeze some layers in VGG16 for fine-tuning
for layer in base_model.layers[-10:]:  # Unfreeze the last 4 layers
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
fine_tune_epochs = 5
history_fine_tune = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=fine_tune_epochs
)

# Step 7: Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Save the model
model.save("brain_tumor_classifier_vgg16_finetuned.h5")


Streaming output truncated to the last 5000 lines.
Processed Tr-me_1023.jpg with median filter
Processed Tr-me_0095.jpg with median filter
Processed Tr-me_0131.jpg with median filter
Processed Tr-me_0059.jpg with median filter
Processed Tr-me_1243.jpg with median filter
Processed Tr-me_1013.jpg with median filter
Processed Tr-me_0641.jpg with median filter
Processed Tr-me_1031.jpg with median filter
Processed Tr-me_0128.jpg with median filter
Processed Tr-me_0097.jpg with median filter
Processed Tr-me_0632.jpg with median filter
Processed Tr-me_1044.jpg with median filter
Processed Tr-me_0248.jpg with median filter
Processed Tr-me_0773.jpg with median filter
Processed Tr-me_0338.jpg with median filter
Processed Tr-me_0918.jpg with median filter
Processed Tr-me_0839.jpg with median filter
Processed Tr-me_1274.jpg with median filter
Processed Tr-me_0602.jpg with median filter
Processed Tr-me_0090.jpg with median filter
Processed Tr-me_0992.jpg with median filter
Processed Tr-me_0325.jpg 

Test Accuracy: 0.97


In [23]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import BatchNormalization


# Apply Median Filter for image enhancement
def apply_median_filter_to_images(input_dir, output_dir, filter_size=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Process each folder inside the input directory (glioma, meningioma, pituitary, no_tumor)
    for class_name in os.listdir(input_dir):
        class_folder = os.path.join(input_dir, class_name)

        # Skip files that are not directories (e.g., test/train folders)
        if not os.path.isdir(class_folder):
            continue

        class_output_folder = os.path.join(output_dir, class_name)
        if not os.path.exists(class_output_folder):
            os.makedirs(class_output_folder)

        # Process each image in the class folder
        for image_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_name)

            # Read and apply median filter
            image = cv2.imread(image_path)
            filtered_image = cv2.medianBlur(image, filter_size)

            # Save the enhanced image
            output_image_path = os.path.join(class_output_folder, image_name)
            cv2.imwrite(output_image_path, filtered_image)
            print(f"Processed {image_name} with median filter")

# Apply median filter to training images
input_dir = os.path.join(extract_path, 'Training')
output_dir = os.path.join(extract_path, 'training_filtered')
apply_median_filter_to_images(input_dir, output_dir)

# Apply median filter to testing images
input_dir = os.path.join(extract_path, 'Testing')
output_dir = os.path.join(extract_path, 'testing_filtered')
apply_median_filter_to_images(input_dir, output_dir)

# Step 3: Prepare data generators for training and testing
image_size = (224, 224)  # VGG16 expects images of size 224x224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255,           # Normalize pixel values
    rotation_range=15,         # Randomly rotate images
    width_shift_range=0.1,     # Shift images horizontally
    height_shift_range=0.1,    # Shift images vertically
    zoom_range=0.1,            # Randomly zoom images
    horizontal_flip=True,      # Randomly flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load the filtered training and testing datasets
train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(extract_path, 'training_filtered'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multiclass classification
)

test_generator = test_datagen.flow_from_directory(
    directory=os.path.join(extract_path, 'testing_filtered'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multiclass classification
)

# Load the VGG16 model (without the top classification layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# Build the complete model with a custom classifier
model = Sequential([
    base_model,                # VGG16 as feature extractor
    Flatten(),                 # Flatten the output of VGG16
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),  # Dense layer for learning high-level features
    Dropout(0.5),              # Dropout for regularization
    Dense(4, activation='softmax')  # Output layer for 4 classes (glioma, meningioma, pituitary, no_tumor)
])

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the model (only the custom layers)
epochs = 10
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs
)

# Unfreeze some layers in VGG16 for fine-tuning
for layer in base_model.layers[-10:]:  # Unfreeze the last 4 layers
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=RMSprop(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
fine_tune_epochs = 5
history_fine_tune = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=fine_tune_epochs
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Save the model
model.save("brain_tumor_classifier_vgg16_finetuned.h5")


Streaming output truncated to the last 5000 lines.
Processed Tr-me_1023.jpg with median filter
Processed Tr-me_0095.jpg with median filter
Processed Tr-me_0131.jpg with median filter
Processed Tr-me_0059.jpg with median filter
Processed Tr-me_1243.jpg with median filter
Processed Tr-me_1013.jpg with median filter
Processed Tr-me_0641.jpg with median filter
Processed Tr-me_1031.jpg with median filter
Processed Tr-me_0128.jpg with median filter
Processed Tr-me_0097.jpg with median filter
Processed Tr-me_0632.jpg with median filter
Processed Tr-me_1044.jpg with median filter
Processed Tr-me_0248.jpg with median filter
Processed Tr-me_0773.jpg with median filter
Processed Tr-me_0338.jpg with median filter
Processed Tr-me_0918.jpg with median filter
Processed Tr-me_0839.jpg with median filter
Processed Tr-me_1274.jpg with median filter
Processed Tr-me_0602.jpg with median filter
Processed Tr-me_0090.jpg with median filter
Processed Tr-me_0992.jpg with median filter
Processed Tr-me_0325.jpg 

Test Accuracy: 0.97
